In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [1]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'

#start Glue
# app = GlueApplication(dc)

# imageviewer = app.new_data_viewer(ImageViewer)
# imageviewer.add_data(image)

# app.start()



In [2]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [3]:
# Constructs a pandas DataFrame instead of an astropy table

import pandas as pd
from pandas import DataFrame
import numpy as np

In [4]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc subset1



In [24]:
type(dc[0])

glue.core.data.Data

In [22]:
array1 = [1,3,5,6]
array2 = [4,5,6,6]
array3 = [6,7,8,9]

colnames = ['array1', 'array2', 'array3']
rownames = ['col1', 'col2', 'col3', 'col4']

data = np.asarray([array1, array2, array3]).transpose()

pandas_data = pd.DataFrame(data, index=names, columns=heads)

In [32]:
# Full working example! 
# Has a dropdown menu with the components- highlights the row of the component selected
# Still need to figure out how to resize window to fit the contents
# Merged into one class and takes in a data object as a parameter

import sys
from PyQt5 import QtCore, QtWidgets
Qt = QtCore.Qt
from PyQt5.QtCore import QVariant
from qtpy.QtWidgets import (QApplication, QLabel, QWidget, QComboBox,
                            QVBoxLayout, QPushButton, QGridLayout, QTableView) 

class stat_widget(QWidget):
    def __init__(self, core_data, parent = None):
        super(stat_widget, self).__init__(parent)
        self.view = QtCore.QAbstractTableModel.__init__(self, parent)
        self.core = core_data       
      
        mean_array = []
        median_array = []
        min_array = []
        max_array = []
        sum_array = []
        name_array = []
        headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

        for j in range(0, len(self.core.components)):
            name = self.core.components[j].label # Get the name of each component
            name_array.append(name) # add to the name array to build the table
            mean_array.append(self.core.compute_statistic('mean', self.core.components[j]))
            median_array.append(self.core.compute_statistic('median', self.core.components[j]))       
            min_array.append(self.core.compute_statistic('minimum', self.core.components[j]))       
            max_array.append(self.core.compute_statistic('maximum', self.core.components[j]))      
            sum_array.append(self.core.compute_statistic('sum', self.core.components[j]))         

            # At this point we have several column arrays to put into a pandas data frame
    
        column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

        self.data_frame = pd.DataFrame(column_data, index=name_array, columns=headings)         
        
        layout = QVBoxLayout()
        self.cb = QComboBox()
    
        for i in range(0, len(self.data_frame.index)):
            self.cb.addItem(self.data_frame.index[i])
        
        self.cb.currentIndexChanged.connect(self.selectionchange)
    
        layout.addWidget(self.cb)  
    
        self.view = QTableView()
        
        # Why does this line work? 
        model = PandasModel(self.data_frame)
        self.view.setModel(model)
        self.view.setShowGrid(False)

        layout.addWidget(self.view) 

        self.setLayout(layout)
        self.setWindowTitle("Statistics")          
        
    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()          
        
    def selectionchange(self,i):
        print ("Current index",i,"selection changed to",self.cb.currentText())
        self.view.selectRow(i)

app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
    
ex = stat_widget(dc[0])
ex.show()
sys.exit(app.exec_())

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x1513880ca8>


SystemExit: 0

/Users/laurachapman/anaconda/envs/glueviz-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
help(QVBoxLayout)

Help on class QVBoxLayout in module PyQt5.QtWidgets:

class QVBoxLayout(QBoxLayout)
 |  QVBoxLayout()
 |  QVBoxLayout(QWidget)
 |  
 |  Method resolution order:
 |      QVBoxLayout
 |      QBoxLayout
 |      QLayout
 |      PyQt5.QtCore.QObject
 |      QLayoutItem
 |      sip.wrapper
 |      sip.simplewrapper
 |      builtins.object
 |  
 |  Methods inherited from QBoxLayout:
 |  
 |  addItem(...)
 |      addItem(self, QLayoutItem)
 |  
 |  addLayout(...)
 |      addLayout(self, QLayout, stretch: int = 0)
 |  
 |  addSpacerItem(...)
 |      addSpacerItem(self, QSpacerItem)
 |  
 |  addSpacing(...)
 |      addSpacing(self, int)
 |  
 |  addStretch(...)
 |      addStretch(self, stretch: int = 0)
 |  
 |  addStrut(...)
 |      addStrut(self, int)
 |  
 |  addWidget(...)
 |      addWidget(self, QWidget, stretch: int = 0, alignment: Union[Qt.Alignment, Qt.AlignmentFlag] = 0)
 |  
 |  count(...)
 |      count(self) -> int
 |  
 |  direction(...)
 |      direction(self) -> QBoxLayout.Directio